In [ ]:
#Trabajo final
#Sandra España


install.packages("e1071")
install.packages("kernlab")
install.packages("ROCR")

In [1]:
library(prob)
library(dplyr)
library(tidyverse)
library(readr)
library(e1071)
library(kernlab)
library(ROCR)

Loading required package: combinat


Attaching package: ‘combinat’


The following object is masked from ‘package:utils’:

    combn


Loading required package: fAsianOptions

Loading required package: timeDate

Loading required package: timeSeries

Loading required package: fBasics

Loading required package: fOptions


Attaching package: ‘prob’


The following objects are masked from ‘package:base’:

    intersect, setdiff, union



Attaching package: ‘dplyr’


The following objects are masked from ‘package:prob’:

    intersect, setdiff, union


The following objects are masked from ‘package:timeSeries’:

    filter, lag


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.

ERROR: Error in library(e1071): there is no package called ‘e1071’


In [2]:
datos<-read.csv(file="enoen_sdemt321.csv",header=T)
#para este trabajo se utilizaron los datos de la Encuesta Nacional de Ocupación y Empleo publicado por el
#Instituto Nacional de Estadística y Geografía (INEGI) al terer trimestre de 2021.
#Esta base de datos ofrece una cobertura nacional de información de empleo, remuneraciones, 
#y otras características relacionadas a nivel hogar, comunidad e individuo.
#Específicamente, se utilizó la base de datos a nivel individual de 428,161 participantes, en la cual
#se reflejan 114 variables como nivel educativo, acceso a seguridad social, remuneración, características 
#del hogar, sexo, edad, localidad, sector de ocupación, condición de ocupación, entre otros.

#No obstante, para efectos de esta investigación, se redujo el conjunto de datos a 15 variables y a 316,407 encuestas completadas de encuestados en edad de trabajar

In [3]:
#limpieza de datos
datos<-data.frame(datos)
datos1<-subset(datos,r_def==0)#depuración1: se depuraron los datos para eliminar entrevistas no completas. Esto es, r_def=0
prueba1<-sum(datos1$r_def)==0 #prueba de que depuración1 se realizó correctamente
datos1<-subset(datos1,eda>15) #depuración2:se depuraron los encuestados que no están en edad de trabajar
                                #el conjunto de datos final comprende 18 datos descritos en la variable dict
prueba1<-sum(datos1$eda)==0 #prueba de que depuración2 se realizó correctamente
datos1<-subset(datos1,clase1!=0)#depuración3: se depuraron los datos donde no está especificado si es PEA oPNEA

head1<-c("PEA","pob_ocupada","entidad","sexo","edad","escolaridad_grado","edo_civil","num_hijos","con_experiencia","subocupado","horas_trabajadas","ingreso_mensual","seguridad_social","actividad_SCIAN","sector_informal")
datos3<-select(datos1,PEA=clase1,pob_ocupada=clase3,entidad=ent,sexo=sex,
               edad=eda,escolaridad_gdo=cs_p13_1,edo_conyugal=e_con,
               num_hijos=n_hij,con_experiencia=d_ant_lab,subocupado=sub_o,
               horas_trabajadas=hrsocup,ingreso_mensual=ingocup,
               seguridad_social=imssissste,actividad_SCIAN=scian,
               sector_informal=emp_ppal)
datos3$PEA[datos3$PEA==2]<-0
datos3$sexo[datos3$sexo==2]<-"mujer"
datos3$sexo[datos3$sexo==1]<-"hombre"
datos3$edo_conyugal[datos3$edo_conyugal==1]<-"casado"
datos3$edo_conyugal[datos3$edo_conyugal==5]<-"casado"
datos3$edo_conyugal[datos3$edo_conyugal==2]<-"no casado"
datos3$edo_conyugal[datos3$edo_conyugal==3]<-"no casado"
datos3$edo_conyugal[datos3$edo_conyugal==4]<-"no casado"
datos3$edo_conyugal[datos3$edo_conyugal==6]<-"no casado"
datos3$edo_conyugal[datos3$edo_conyugal==9]<-"no casado"


In [4]:
#Probabilidad de pertenecer a la PEA
PEA11<-data.frame(table(datos3$PEA))
PEA12<-probspace(PEA11[,1],PEA11[,2])
PEA12 #la probabilidad de que un individuo de la muestra pertenezca a la PEA es de 61.45%

x,probs
<fct>,<dbl>
0,0.3823445
1,0.6176555


In [5]:
#Probabilidad de estar ocupado
labels1<-c("0:PNEA","1:ocupado pleno","2: ocupado sin pago",
           "3: ocupado ausente con nexo laboral","4:ocupado ausente con retorno asegurado",
           "5: desocupado iniciador","6: desocupado con búsqueda","7:desocupados ausentes sin ingreso y sin nexo")
ocupado11<-data.frame(table(datos3$pob_ocupada))
ocupado12<-probspace(ocupado11[2:8,1],ocupado11[2:8,2])
data.frame(labels1[2:8],ocupado12)
pobocupada<-Prob(ocupado12[1:4,])
pobdesocupada<-Prob(ocupado12[5:7,])
pobocupada
pobdesocupada
#la probabilidad de estar ocupado fue de 96.1%, por lo que la tasa de desmpleo fue de 3.91%
setocupado<-probspace(ocupado11[2:5,1],ocupado11[2:5,2])
data.frame(labels1[2:5],setocupado)
ocupado1<-datos3$pob_ocupada!=0
ocupado2<-datos3$pob_ocupada>=5
datos3$ocupado<-ocupado1+ocupado2
summary(datos3)
#dado el conjunto de población ocupada, la probabilidad deser ocupado pleno es del 93%, mientras que ser ocupado sin pago tenía una probabilidad de 2.1

labels1.2.8.,x,probs
<chr>,<fct>,<dbl>
1:ocupado pleno,1,0.9025567755
2: ocupado sin pago,2,0.0194465333
3: ocupado ausente con nexo laboral,3,0.0260640698
4:ocupado ausente con retorno asegurado,4,0.0088434351
5: desocupado iniciador,5,0.0007369529
6: desocupado con búsqueda,6,0.0413896827
7:desocupados ausentes sin ingreso y sin nexo,7,0.0009625508


[1] 0.9569108

[1] 0.04308919

labels1.2.5.,x,probs
<chr>,<fct>,<dbl>
1:ocupado pleno,1,0.94319843
2: ocupado sin pago,2,0.02032220
3: ocupado ausente con nexo laboral,3,0.02723772
4:ocupado ausente con retorno asegurado,4,0.00924165


      PEA          pob_ocupada        entidad          sexo          
 Min.   :0.0000   Min.   :0.0000   Min.   : 1.00   Length:107649     
 1st Qu.:0.0000   1st Qu.:0.0000   1st Qu.: 9.00   Class :character  
 Median :1.0000   Median :1.0000   Median :15.00   Mode  :character  
 Mean   :0.6177   Mean   :0.8115   Mean   :16.28                     
 3rd Qu.:1.0000   3rd Qu.:1.0000   3rd Qu.:24.00                     
 Max.   :1.0000   Max.   :7.0000   Max.   :32.00                     
                                                                     
      edad       escolaridad_gdo  edo_conyugal         num_hijos    
 Min.   :16.00   Min.   : 0.000   Length:107649      Min.   : 0.00  
 1st Qu.:27.00   1st Qu.: 3.000   Class :character   1st Qu.: 0.00  
 Median :40.00   Median : 4.000   Mode  :character   Median : 2.00  
 Mean   :42.09   Mean   : 4.071                      Mean   : 2.23  
 3rd Qu.:55.00   3rd Qu.: 6.000                      3rd Qu.: 3.00  
 Max.   :98.00   Max.   :9

In [6]:
#Probabilidades por entidad
abrev=c("","AGS","BCN","BCS","CAMP","COAH","COL","CHIS","CHIH","CDMX","DUR","GTO","GRO","HGO","JAL","EDOMEX","MICH","MOR","NAY","NL","OAX","PUE","QRO","QROO","SLP","SIN","SON","TAB","TAMP","TLAX","VER","YUC","ZAC")
#NOTA IMPORTANTE: la nota metodológica de INEGI tiene un error en la página 7: los estados NL, PUE,QRO,QROO,SLP se repiten, por lo que en este trabajo se tiene que replicar este error al no tener la información correcta de qué estados tratan ver https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/doc/enoe_n_fd_c_bas_amp.pdf
entidad11<-data.frame(table(datos3$entidad,datos3$ocupado))
entidad22<-data.frame(table(datos3$entidad,datos3$pob_ocupada))
prob11<-probspace(entidad11[,1:2],entidad11[,3])
entidad12<-matrix(0,3,32) #PNEA, PEA ocupado o PEA desocupado
entidad13<-matrix(0,2,32) #PEA ocupado o PEA desocupado
entidad14<-matrix(0,8,32) #PNEA o PEA por 7 tipos (ver labels1)
entidad15<-matrix(0,4,32) #ocupados por tipo
entidad16<-matrix(0,3,32) #desocupados por tipo
for (i in 1:32) {
estado1<-subset(entidad11,Var1==i)
prob12<-probspace(estado1[,2],estado1[,3])
entidad12[,i]<-prob12$probs
estado2<-subset(estado1,Var2!=0)
prob13<-probspace(estado2[,2],estado2[,3])
entidad13[,i]<-prob13$probs
estado3<-subset(entidad22,Var1==i)
prob14<-probspace(estado3[,2],estado3[,3])
entidad14[,i]<-prob14$probs
estado4<-estado3[2:5,]
prob15<-probspace(estado4[,2],estado4[,3])
entidad15[,i]<-prob15$probs
estado5<-estado3[6:8,]
prob16<-probspace(estado5[,2],estado5[,3])
entidad16[,i]<-prob16$probs}                  

#probabilidad de pertenecer a la PEA/PNEA
pneaypea<-data.frame(c("PNEA","PEA ocupado","PEA desocupado"),entidad12) #probabilidad de pertenecer a la PEA y estar ocupado (1), pertenecer a la PEA y estar desocupado(2) o no pertenecer a la PEA por estado
colnames(pneaypea)<-abrev
pneaypea
#probabilidad de pertenecer a la PEA/PNEA por rubro
peagral<-data.frame(labels1,entidad14)
colnames(peagral)<-abrev
peagral
#probabilidad de estar ocupado/desocupado dado que pertenece a la PEA
peaporestado<-data.frame(c("ocupados","desocupados"),entidad13) #probabilidad de estar ocupado (1) o desocupado (2) por estado dado que pertenece a la PEA
colnames(peaporestado)<-abrev
peaporestado
#tipos de ocupados dado que está ocupado
peaocupada<-data.frame(labels1[2:5],entidad15)
colnames(peaocupada)<-abrev
peaocupada
#tipos de desocupados dado que está desocupado
peadesocupada<-data.frame(labels1[6:8],entidad16)
colnames(peadesocupada)<-abrev
peadesocupada

#prueba que datos3$ocupado no sean datos sin información sino que sea PNEA
pnea<-subset(datos3,PEA==0)
ocupado0<-subset(datos3,ocupado!=0)
pruebapnea<-intersect(pnea,ocupado0) #se verificó que no hubiera ocupados/desocupados que pertenecieran a la pnea. Esto es, que efectivamente datos3$ocupado=0 fuera el conjunto de la pnea y no valores no capturados
pruebapnea
dim(pruebapnea)

,AGS,BCN,BCS,CAMP,COAH,COL,CHIS,CHIH,CDMX,⋯,QROO,SLP,SIN,SON,TAB,TAMP,TLAX,VER,YUC,ZAC
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
PNEA,0.4113093,0.38229755,0.3427371,0.37620579,0.35923336,0.29995067,0.42203742,0.36722050,0.37707787,⋯,0.3639906,0.37572703,0.39642120,0.35361781,0.41982609,0.37951657,0.34847979,0.42794760,0.33762772,0.40060140
PEA ocupado,0.5627735,0.59981168,0.6302521,0.59914255,0.61279461,0.68426246,0.55010395,0.61362996,0.57786527,⋯,0.6091193,0.59402869,0.58258775,0.61892393,0.54260870,0.59406927,0.61677247,0.54792002,0.64326966,0.57500835
PEA desocupado,0.0259172,0.01789077,0.0270108,0.02465166,0.02797203,0.01578688,0.02785863,0.01914954,0.04505687,⋯,0.0268901,0.03024428,0.02099105,0.02745826,0.03756522,0.02641415,0.03474774,0.02413238,0.01910262,0.02439024


,AGS,BCN,BCS,CAMP,COAH,COL,CHIS,CHIH,CDMX,⋯,QROO,SLP,SIN,SON,TAB,TAMP,TLAX,VER,YUC,ZAC
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0:PNEA,0.411309323,0.3822975518,0.3427370948,0.3762057878,0.3592333592,0.2999506660,0.4220374220,0.3672205013,0.377077865,⋯,0.3639906469,0.375727026,0.396421198,0.3536178108,0.4198260870,0.3795165711,0.3484797862,0.4279475983,0.337627721,0.4006014033
1:ocupado pleno,0.553012454,0.5737602009,0.5720288115,0.5495712755,0.5723905724,0.6364084854,0.5259875260,0.5806814982,0.553805774,⋯,0.5678098207,0.547499031,0.552993806,0.5684601113,0.5012173913,0.5517069524,0.5659873037,0.5189611584,0.603731675,0.5282325426
2: ocupado sin pago,0.001346348,0.0091023227,0.0144057623,0.0182207931,0.0126910127,0.0207202763,0.0079002079,0.0056322163,0.004155731,⋯,0.0241621200,0.025979062,0.001032347,0.0137291280,0.0163478261,0.0159481685,0.0354159706,0.0103424500,0.012883163,0.0193785499
3: ocupado ausente con nexo laboral,0.006395153,0.0131826742,0.0342136855,0.0217041801,0.0230510231,0.0192402565,0.0118503119,0.0205575894,0.014873141,⋯,0.0116913484,0.014346646,0.025808672,0.0319109462,0.0201739130,0.0201844007,0.0080187103,0.0114916111,0.016881386,0.0210491146
4:ocupado ausente con retorno asegurado,0.002019522,0.0037664783,0.0096038415,0.0096463023,0.0046620047,0.0078934386,0.0043659044,0.0067586595,0.005030621,⋯,0.0054559626,0.006203955,0.002752925,0.0048237477,0.0048695652,0.0062297533,0.0073504845,0.0071247989,0.009773434,0.0063481457
5: desocupado iniciador,0.000336587,0.0012554928,0.0006002401,0.0005359057,0.0002590003,0.0004933399,0.0008316008,0.0000000000,0.000000000,⋯,0.0003897116,0.000000000,0.001376462,0.0003710575,0.0010434783,0.0004983803,0.0003341129,0.0004596644,0.000444247,0.0006682259
6: desocupado con búsqueda,0.024907439,0.0163214062,0.0252100840,0.0235798499,0.0277130277,0.0148001973,0.0268191268,0.0185863137,0.042869641,⋯,0.0257209665,0.028693292,0.019614591,0.0263450835,0.0358260870,0.0251682033,0.0337454059,0.0229832222,0.018658374,0.0223855663
7:desocupados ausentes sin ingreso y sin nexo,0.000673174,0.0003138732,0.0012004802,0.0005359057,0.0000000000,0.0004933399,0.0002079002,0.0005632216,0.002187227,⋯,0.0007794232,0.001550989,0.000000000,0.0007421150,0.0006956522,0.0007475704,0.0006682259,0.0006894967,0.000000000,0.0013364517


,AGS,BCN,BCS,CAMP,COAH,COL,CHIS,CHIH,CDMX,⋯,QROO,SLP,SIN,SON,TAB,TAMP,TLAX,VER,YUC,ZAC
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ocupados,0.95597484,0.97103659,0.95890411,0.9604811,0.956346,0.97744891,0.95179856,0.96973743,0.92766854,⋯,0.95772059,0.9515528,0.96522235,0.95752009,0.9352518,0.95742972,0.94666667,0.95781438,0.9711603,0.95930881
desocupados,0.04402516,0.02896341,0.04109589,0.0395189,0.043654,0.02255109,0.04820144,0.03026257,0.07233146,⋯,0.04227941,0.0484472,0.03477765,0.04247991,0.0647482,0.04257028,0.05333333,0.04218562,0.0288397,0.04069119


,AGS,BCN,BCS,CAMP,COAH,COL,CHIS,CHIH,CDMX,⋯,QROO,SLP,SIN,SON,TAB,TAMP,TLAX,VER,YUC,ZAC
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1:ocupado pleno,0.982655502,0.956567242,0.90761905,0.91726297,0.934065934,0.93006489,0.956160242,0.946305645,0.958364875,⋯,0.932181702,0.92167102,0.949202599,0.918465228,0.923717949,0.92869128,0.91765980,0.94714765,0.93853591,0.91865195
2: ocupado sin pago,0.002392344,0.015175301,0.02285714,0.03041145,0.020710059,0.03028118,0.014361300,0.009178522,0.007191522,⋯,0.039667306,0.04373368,0.001772002,0.022182254,0.030128205,0.02684564,0.05742145,0.01887584,0.02002762,0.03370134
3: ocupado ausente con nexo laboral,0.011363636,0.021978022,0.05428571,0.03622540,0.037616230,0.02811824,0.021541950,0.033501606,0.025738077,⋯,0.019193858,0.02415144,0.044300059,0.051558753,0.037179487,0.03397651,0.01300108,0.02097315,0.02624309,0.03660662
4:ocupado ausente con retorno asegurado,0.003588517,0.006279435,0.01523810,0.01610018,0.007607777,0.01153569,0.007936508,0.011014227,0.008705526,⋯,0.008957134,0.01044386,0.004725340,0.007793765,0.008974359,0.01048658,0.01191766,0.01300336,0.01519337,0.01104009


,AGS,BCN,BCS,CAMP,COAH,COL,CHIS,CHIH,CDMX,⋯,QROO,SLP,SIN,SON,TAB,TAMP,TLAX,VER,YUC,ZAC
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
5: desocupado iniciador,0.01298701,0.07017544,0.02222222,0.02173913,0.009259259,0.03125,0.029850746,0.00000000,0.00000000,⋯,0.01449275,0.00000000,0.06557377,0.01351351,0.02777778,0.01886792,0.009615385,0.01904762,0.02325581,0.02739726
6: desocupado con búsqueda,0.96103896,0.91228070,0.93333333,0.95652174,0.990740741,0.93750,0.962686567,0.97058824,0.95145631,⋯,0.95652174,0.94871795,0.93442623,0.95945946,0.95370370,0.95283019,0.971153846,0.95238095,0.97674419,0.91780822
7:desocupados ausentes sin ingreso y sin nexo,0.02597403,0.01754386,0.04444444,0.02173913,0.000000000,0.03125,0.007462687,0.02941176,0.04854369,⋯,0.02898551,0.05128205,0.00000000,0.02702703,0.01851852,0.02830189,0.019230769,0.02857143,0.00000000,0.05479452


PEA,pob_ocupada,entidad,sexo,edad,escolaridad_gdo,edo_conyugal,num_hijos,con_experiencia,subocupado,horas_trabajadas,ingreso_mensual,seguridad_social,actividad_SCIAN,sector_informal,ocupado
<dbl>,<int>,<int>,<chr>,<int>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>


[1]  0 16

In [7]:
#Probabilidades por entidad por sexo
entidad33<-data.frame(table(datos3$entidad,datos3$ocupado,datos3$sexo))
sexo11<-matrix(0,6,32)
sexo12<-matrix(0,6,32)
sexo13<-matrix(0,4,32)
for (i in 1:32) {
#probabilidad de ser económicamente activo por sexo
sexo1<-subset(entidad33,Var1==i)
prob21<-probspace(data.frame(sexo1[,2],sexo1[,3]),sexo1[,4])
sexo11[,i]<-prob21$probs
#probabilidad de estar ocupado por sexo
sexo2<-subset(sexo1,Var3=="hombre")
sexo3<-subset(sexo1,Var3=="mujer")
prob22<-probspace(sexo2[,2],sexo2[,4])
prob23<-probspace(sexo3[,2],sexo3[,4])
sexo12[1:3,i]<-prob22$probs   
sexo12[4:6,i]<-prob23$probs
#probabilidad de estar desocupado por sexo
sexo4<-subset(sexo2,Var2!=0) #PEA hombres
sexo5<-subset(sexo3,Var2!=0) #PEA mujeres
prob24<-probspace(sexo4[,2],sexo4[,4])
prob25<-probspace(sexo5[,2],sexo5[,4])
sexo13[1:2,i]<-prob24$probs
sexo13[3:4,i]<-prob25$probs
}
colnames(sexo11)<-abrev[2:33]
colnames(sexo12)<-abrev[2:33]
colnames(sexo13)<-abrev[2:33]
ocupacion=c("PNEA","PEA ocupado","PEA desocupado")
data.frame(data.frame(ocupacion,sexo=sexo1[,3]),sexo11)
data.frame(data.frame(ocupacion,sexo=sexo1[,3]),sexo12)
data.frame(data.frame(estatus_PEA=c("ocupado","desocupado"),sexo=c("hombre","hombre","mujer","mujer")),sexo13)

ocupacion,sexo,AGS,BCN,BCS,CAMP,COAH,COL,CHIS,CHIH,⋯,QROO,SLP,SIN,SON,TAB,TAMP,TLAX,VER,YUC,ZAC
<chr>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
PNEA,hombre,0.108717604,0.120213434,0.10924370,0.095391211,0.095830096,0.094227923,0.09812890,0.108983385,⋯,0.09469992,0.09965103,0.120096352,0.10129870,0.11165217,0.103164715,0.09021049,0.11629510,0.090182141,0.10758436
PEA ocupado,hombre,0.337596769,0.366290019,0.38715486,0.353161844,0.375032375,0.383818451,0.33596674,0.370318220,⋯,0.39360873,0.34470725,0.344803854,0.35992579,0.33252174,0.347620234,0.34914801,0.33164790,0.391381608,0.33678583
PEA desocupado,hombre,0.018848872,0.009730069,0.01260504,0.015809218,0.018648019,0.009373458,0.01413721,0.010701211,⋯,0.01675760,0.01667313,0.011699931,0.01669759,0.02434783,0.017941690,0.01670565,0.01401977,0.007996446,0.01236218
PNEA,mujer,0.302591720,0.262084118,0.23349340,0.280814577,0.263403263,0.205722743,0.32390852,0.258237116,⋯,0.26929072,0.27607600,0.276324845,0.25231911,0.30817391,0.276351856,0.25826930,0.31165249,0.247445580,0.29301704
PEA ocupado,mujer,0.225176708,0.233521657,0.24309724,0.245980707,0.237762238,0.300444006,0.21413721,0.243311743,⋯,0.21551052,0.24932144,0.237783895,0.25899814,0.21008696,0.246449041,0.26762446,0.21627212,0.251888050,0.23822252
PEA desocupado,mujer,0.007068327,0.008160703,0.01440576,0.008842444,0.009324009,0.006413419,0.01372141,0.008448324,⋯,0.01013250,0.01357115,0.009291122,0.01076067,0.01321739,0.008472464,0.01804210,0.01011262,0.011106175,0.01202807


ocupacion,sexo,AGS,BCN,BCS,CAMP,COAH,COL,CHIS,CHIH,⋯,QROO,SLP,SIN,SON,TAB,TAMP,TLAX,VER,YUC,ZAC
<chr>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
PNEA,hombre,0.23371925,0.24225174,0.21462264,0.20542412,0.19576720,0.19331984,0.21892393,0.22241379,⋯,0.18750000,0.21614802,0.25198556,0.21195652,0.23830735,0.22009569,0.19780220,0.25174129,0.18421053,0.23555230
PEA ocupado,hombre,0.72575977,0.73814042,0.76061321,0.76053087,0.76613757,0.78744939,0.74953618,0.75574713,⋯,0.77932099,0.74768713,0.72346570,0.75310559,0.70972532,0.74162679,0.76556777,0.71791045,0.79945554,0.73738113
PEA desocupado,hombre,0.04052098,0.01960784,0.02476415,0.03404501,0.03809524,0.01923077,0.03153989,0.02183908,⋯,0.03317901,0.03616484,0.02454874,0.03493789,0.05196733,0.03827751,0.03663004,0.03034826,0.01633394,0.02706657
PNEA,mujer,0.56576463,0.52024922,0.47555012,0.52426213,0.51598174,0.40134745,0.58703843,0.50635008,⋯,0.54409449,0.51223022,0.52794214,0.48329780,0.57984293,0.52016886,0.47481572,0.57923964,0.48476936,0.53936039
PEA ocupado,mujer,0.42101951,0.46355140,0.49511002,0.45922961,0.46575342,0.58614052,0.38809344,0.47708448,⋯,0.43543307,0.46258993,0.45430638,0.49609097,0.39528796,0.46388368,0.49201474,0.40196497,0.49347258,0.43849938
PEA desocupado,mujer,0.01321586,0.01619938,0.02933985,0.01650825,0.01826484,0.01251203,0.02486812,0.01656543,⋯,0.02047244,0.02517986,0.01775148,0.02061123,0.02486911,0.01594747,0.03316953,0.01879539,0.02175805,0.02214022


estatus_PEA,sexo,AGS,BCN,BCS,CAMP,COAH,COL,CHIS,CHIH,⋯,QROO,SLP,SIN,SON,TAB,TAMP,TLAX,VER,YUC,ZAC
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ocupado,hombre,0.94711992,0.97412354,0.96846847,0.95715323,0.95263158,0.97616060,0.95961995,0.97191426,⋯,0.95916429,0.95386266,0.96718147,0.95566502,0.93177388,0.95092025,0.95433790,0.95944149,0.97997775,0.96459330
desocupado,hombre,0.05288008,0.02587646,0.03153153,0.04284677,0.04736842,0.02383940,0.04038005,0.02808574,⋯,0.04083571,0.04613734,0.03281853,0.04433498,0.06822612,0.04907975,0.04566210,0.04055851,0.02002225,0.03540670
ocupado,mujer,0.96956522,0.96623377,0.94405594,0.96529968,0.96226415,0.97909968,0.93978102,0.96644295,⋯,0.95509499,0.94837758,0.96239554,0.96011004,0.94080997,0.96676442,0.93684211,0.95532995,0.95777027,0.95193591
desocupado,mujer,0.03043478,0.03376623,0.05594406,0.03470032,0.03773585,0.02090032,0.06021898,0.03355705,⋯,0.04490501,0.05162242,0.03760446,0.03988996,0.05919003,0.03323558,0.06315789,0.04467005,0.04222973,0.04806409


In [45]:
escolaridad=c("nivel escolar","preescolar","primaria","secundaria","preparatoria","normal","carrera técnica","profesional","maestría","doctorado","no sabe")
escolaridadpnea<-matrix(0,11,32)
escolaridadpeaocup<-matrix(0,11,32)
escolaridadpeanocup<-matrix(0,11,32)
entidad44<-data.frame(table(datos3$entidad,datos3$ocupado,datos3$escolaridad_gdo))
escolaridad2<-probspace(data.frame(entidad=entidad44[,1],ocupado=entidad44[,2],escolaridad=entidad44[,3]),entidad44[,4])
for(i in 1:32){s1<-subset(escolaridad2,entidad==i)
               s2<-subset(s1,ocupado==0)
               s3<-subset(s1,ocupado==1)
               s4<-subset(s1,ocupado==2)
               pr01<-probspace(s2,s2$probs)
               pr02<-probspace(s3,s3$probs)
               pr03<-probspace(s4,s4$probs)
               escolaridadpnea[,i]<-pr01$probs
               escolaridadpeaocup[,i]<-pr02$probs
               escolaridadpeanocup[,i]<-pr02$probs
}
colnames(escolaridadpnea)<-abrev[2:33]
colnames(escolaridadpeaocup)<-abrev[2:33]
colnames(escolaridadpeanocup)<-abrev[2:33]

data.frame(escolaridadpnea=escolaridad,escolaridadpnea)
data.frame(escolaridadpeaocup=escolaridad,escolaridadpeaocup)
data.frame(escolaridadpeanocup=escolaridad,escolaridadpeanocup)

escolaridadpnea,AGS,BCN,BCS,CAMP,COAH,COL,CHIS,CHIH,CDMX,⋯,QROO,SLP,SIN,SON,TAB,TAMP,TLAX,VER,YUC,ZAC
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
nivel escolar,0.0531914894,0.0426929392,0.056042032,0.091880342,0.0374909877,0.047697368,0.1197044335,0.0398773006,0.0313225058,⋯,0.070663812,0.077399381,0.045138889,0.028331584,0.0521955261,0.0387393303,0.0776605944,0.091836735,0.076315789,0.0658882402
preescolar,0.0000000000,0.0008210181,0.000000000,0.000000000,0.0007209805,0.001644737,0.0014778325,0.0015337423,0.0023201856,⋯,0.002141328,0.001031992,0.000000000,0.002098636,0.0000000000,0.0019697965,0.0009587728,0.002148228,0.001315789,0.0008340284
primaria,0.2839607201,0.2159277504,0.217162872,0.230056980,0.2415284787,0.266447368,0.2477832512,0.2354294479,0.2186774942,⋯,0.216274090,0.241486068,0.265625000,0.215110178,0.2651201326,0.2600131320,0.2588686481,0.288399570,0.298684211,0.2752293578
secundaria,0.2684124386,0.2446633826,0.248686515,0.237891738,0.2746935833,0.213815789,0.2182266010,0.2515337423,0.2180974478,⋯,0.268736617,0.275541796,0.186631944,0.253934942,0.2460646230,0.2731451083,0.2636625120,0.211600430,0.214473684,0.2693911593
preparatoria,0.2078559738,0.2668308703,0.264448336,0.220797721,0.2235039654,0.212171053,0.2123152709,0.2039877301,0.2517401392,⋯,0.291220557,0.189886481,0.229166667,0.244491081,0.2245236123,0.2422849639,0.2214765101,0.214285714,0.206578947,0.1659716430
normal,0.0098199673,0.0090311987,0.007005254,0.009971510,0.0115356885,0.021381579,0.0088669951,0.0061349693,0.0069605568,⋯,0.000000000,0.011351909,0.012152778,0.007345226,0.0049710025,0.0039395929,0.0095877277,0.001611171,0.010526316,0.0200166806
carrera técnica,0.0327332242,0.0369458128,0.043782837,0.051282051,0.0620043259,0.052631579,0.0172413793,0.0713190184,0.0620649652,⋯,0.018201285,0.041279670,0.053819444,0.076600210,0.0323115162,0.0308601445,0.0191754554,0.037593985,0.059210526,0.0450375313
profesional,0.1391162029,0.1691297209,0.152364273,0.150997151,0.1449170872,0.172697368,0.1674876847,0.1771472393,0.1948955916,⋯,0.128479657,0.151702786,0.198784722,0.153200420,0.1640430820,0.1431385424,0.1447746884,0.144468314,0.121052632,0.1476230192
maestría,0.0040916530,0.0073891626,0.008756567,0.004273504,0.0028839221,0.008223684,0.0064039409,0.0115030675,0.0092807425,⋯,0.003211991,0.006191950,0.005208333,0.007345226,0.0082850041,0.0039395929,0.0019175455,0.006444683,0.009210526,0.0100083403


escolaridadpeaocup,AGS,BCN,BCS,CAMP,COAH,COL,CHIS,CHIH,CDMX,⋯,QROO,SLP,SIN,SON,TAB,TAMP,TLAX,VER,YUC,ZAC
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
nivel escolar,0.0065789474,0.0141287284,0.011428571,0.0259391771,0.0063398140,0.017303533,0.0483749055,0.0188159706,0.0083270250,⋯,0.035188740,0.0267624021,0.018901359,0.005995204,0.023717949,0.0205536913,0.016793066,0.0398489933,0.037292818,0.0156885532
preescolar,0.0005980861,0.0005232862,0.000000000,0.0004472272,0.0004226543,0.000000000,0.0003779289,0.0004589261,0.0003785011,⋯,0.001919386,0.0006527415,0.000000000,0.001199041,0.000000000,0.0020973154,0.000000000,0.0016778523,0.000000000,0.0000000000
primaria,0.1901913876,0.1418105704,0.132380952,0.1596601073,0.1196111581,0.151405912,0.2082388511,0.1390546122,0.1150643452,⋯,0.144593730,0.1703655352,0.161842882,0.091726619,0.158974359,0.1581375839,0.215601300,0.2239932886,0.198204420,0.1905868681
secundaria,0.3247607656,0.2721088435,0.280952381,0.2634168157,0.3085376162,0.270367700,0.1976568405,0.2849931161,0.2399697199,⋯,0.291106846,0.2813315927,0.200236267,0.273381295,0.239102564,0.2843959732,0.306067172,0.2273489933,0.252071823,0.2760023242
preparatoria,0.2218899522,0.2715855573,0.280000000,0.2495527728,0.2307692308,0.240807498,0.2346938776,0.2280862781,0.2566237699,⋯,0.293666027,0.2265013055,0.239810986,0.251798561,0.266025641,0.2562919463,0.247020585,0.2420302013,0.232734807,0.1801278327
normal,0.0011961722,0.0005232862,0.000000000,0.0017889088,0.0033812342,0.001441961,0.0015117158,0.0000000000,0.0011355034,⋯,0.000000000,0.0019582245,0.000000000,0.000000000,0.000000000,0.0016778523,0.001083424,0.0012583893,0.002071823,0.0029052876
carrera técnica,0.0239234450,0.0245944532,0.023809524,0.0339892665,0.0587489434,0.035328046,0.0117157974,0.0413033502,0.0492051476,⋯,0.016634677,0.0359007833,0.027761370,0.048561151,0.031410256,0.0247483221,0.022751896,0.0163590604,0.051104972,0.0354445090
profesional,0.2117224880,0.2511773940,0.237142857,0.2428443649,0.2552831784,0.252343187,0.2675736961,0.2510325838,0.2895533687,⋯,0.204094690,0.2317232376,0.329592439,0.289568345,0.258974359,0.2369966443,0.178764897,0.2281879195,0.186464088,0.2568274259
maestría,0.0167464115,0.0151753009,0.026666667,0.0174418605,0.0147928994,0.027397260,0.0253212396,0.0307480496,0.0340651022,⋯,0.010236724,0.0215404700,0.018901359,0.028776978,0.017948718,0.0142617450,0.009750813,0.0142617450,0.034530387,0.0360255665


escolaridadpeanocup,AGS,BCN,BCS,CAMP,COAH,COL,CHIS,CHIH,CDMX,⋯,QROO,SLP,SIN,SON,TAB,TAMP,TLAX,VER,YUC,ZAC
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
nivel escolar,0.0065789474,0.0141287284,0.011428571,0.0259391771,0.0063398140,0.017303533,0.0483749055,0.0188159706,0.0083270250,⋯,0.035188740,0.0267624021,0.018901359,0.005995204,0.023717949,0.0205536913,0.016793066,0.0398489933,0.037292818,0.0156885532
preescolar,0.0005980861,0.0005232862,0.000000000,0.0004472272,0.0004226543,0.000000000,0.0003779289,0.0004589261,0.0003785011,⋯,0.001919386,0.0006527415,0.000000000,0.001199041,0.000000000,0.0020973154,0.000000000,0.0016778523,0.000000000,0.0000000000
primaria,0.1901913876,0.1418105704,0.132380952,0.1596601073,0.1196111581,0.151405912,0.2082388511,0.1390546122,0.1150643452,⋯,0.144593730,0.1703655352,0.161842882,0.091726619,0.158974359,0.1581375839,0.215601300,0.2239932886,0.198204420,0.1905868681
secundaria,0.3247607656,0.2721088435,0.280952381,0.2634168157,0.3085376162,0.270367700,0.1976568405,0.2849931161,0.2399697199,⋯,0.291106846,0.2813315927,0.200236267,0.273381295,0.239102564,0.2843959732,0.306067172,0.2273489933,0.252071823,0.2760023242
preparatoria,0.2218899522,0.2715855573,0.280000000,0.2495527728,0.2307692308,0.240807498,0.2346938776,0.2280862781,0.2566237699,⋯,0.293666027,0.2265013055,0.239810986,0.251798561,0.266025641,0.2562919463,0.247020585,0.2420302013,0.232734807,0.1801278327
normal,0.0011961722,0.0005232862,0.000000000,0.0017889088,0.0033812342,0.001441961,0.0015117158,0.0000000000,0.0011355034,⋯,0.000000000,0.0019582245,0.000000000,0.000000000,0.000000000,0.0016778523,0.001083424,0.0012583893,0.002071823,0.0029052876
carrera técnica,0.0239234450,0.0245944532,0.023809524,0.0339892665,0.0587489434,0.035328046,0.0117157974,0.0413033502,0.0492051476,⋯,0.016634677,0.0359007833,0.027761370,0.048561151,0.031410256,0.0247483221,0.022751896,0.0163590604,0.051104972,0.0354445090
profesional,0.2117224880,0.2511773940,0.237142857,0.2428443649,0.2552831784,0.252343187,0.2675736961,0.2510325838,0.2895533687,⋯,0.204094690,0.2317232376,0.329592439,0.289568345,0.258974359,0.2369966443,0.178764897,0.2281879195,0.186464088,0.2568274259
maestría,0.0167464115,0.0151753009,0.026666667,0.0174418605,0.0147928994,0.027397260,0.0253212396,0.0307480496,0.0340651022,⋯,0.010236724,0.0215404700,0.018901359,0.028776978,0.017948718,0.0142617450,0.009750813,0.0142617450,0.034530387,0.0360255665


In [59]:
experiencia=c("con experiencia","sin experiencia")
#los datos de experiencia tienen muchos campos vacíos (3 cuartiles), por lo que los resultados de esta investigación no serían representativos


In [132]:
#es posible que los datos de subocupación también tengan muchos datos vacíos (el 1er y 3er cuartiles son iguales a cero). Sólo 7.7% de la población reporta estar subocupada, lo cual podría plantear dos escenarios 1) la desocupación en el país es muy baja, 2)los datos de subocupación contienen un error de medición
sum(datos3$subocupado)/nrow(datos3)

entidad5<-data.frame(table(datos3$entidad,datos3$subocupado,datos3$sexo))
subocup1<-subset(entidad5,Var2!=0)
subocupados<-matrix(0,2,32)
for(i in 1:32){sub1<-subset(subocup1,Var1==i)
               prs1<-probspace(sub1,sub1$Freq)
               subocupados[,i]=prs1$probs
}
colnames(subocupados)<-abrev[2:33]
data.frame(sexo=c("hombres","mujeres"),subocupados)

#brecha de género por entidad por nivel escolar
entidad6<-data.frame(datos3$entidad,datos3$sexo,datos3$ingreso,datos3$escolaridad_gdo)
for (i in 1:32){A[i]<-count(subset(entidad6,datos3.entidad==i))}
for (i in 1:32){brecha1<-subset(entidad6,entidad6$datos3.entidad==i)
              hombres1<-subset(brecha1,brecha1$datos3.sexo=="hombre")
            mujeres1<-subset(brecha1,brecha1$datos3.sexo=="mujer")
                m1<-matrix(0,11,32)
                for(j in 1:11){escol1<-subset(hombres1,hombres1$escolaridad_gdo==j-1)
                               k1<-mean(escol1$datos3.ingreso)
                               escol2<-subset(mujeres1,mujeres1$escolaridad_gdo==j-1)
                               k2<-mean(escol2$datos3.ingreso)
                               m1[j,i]<-k1/k2-1
                }
               }

[1] 0.07674015

sexo,AGS,BCN,BCS,CAMP,COAH,COL,CHIS,CHIH,CDMX,⋯,QROO,SLP,SIN,SON,TAB,TAMP,TLAX,VER,YUC,ZAC
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
hombres,0.6153846,0.619469,0.60181,0.5857988,0.6963351,0.5447761,0.5661538,0.6363636,0.5877712,⋯,0.5691057,0.6208178,0.5806452,0.5574713,0.6292683,0.6016484,0.5620767,0.6095238,0.6868132,0.6137184
mujeres,0.3846154,0.380531,0.39819,0.4142012,0.3036649,0.4552239,0.4338462,0.3636364,0.4122288,⋯,0.4308943,0.3791822,0.4193548,0.4425287,0.3707317,0.3983516,0.4379233,0.3904762,0.3131868,0.3862816


In [133]:
escol1

datos3.entidad,datos3.sexo,datos3.ingreso,datos3.escolaridad_gdo
<int>,<chr>,<int>,<int>


In [134]:
hombres1

,datos3.entidad,datos3.sexo,datos3.ingreso,datos3.escolaridad_gdo
,<int>,<chr>,<int>,<int>
225,32,hombre,4300,0
226,32,hombre,0,3
274,32,hombre,0,4
338,32,hombre,4300,3
384,32,hombre,7740,4
568,32,hombre,17200,2
630,32,hombre,8600,2
655,32,hombre,0,2
704,32,hombre,13330,2
